# Импорт библиотек

In [1]:
import random
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import warnings
# warnings.filterwarnings('ignore')
import scipy.stats as ss
from typing import Union, List, Dict, Tuple
from sklearn.model_selection import train_test_split

# Конфиг 

In [2]:
with open('config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
path_to_data = config['path_to_data']
seed = config['random_state']

%config InlineBackend.figure_formats = config['backend_figure_formats']
random.seed(seed)
np.random.seed(seed)
# pd.set_option('display.precision', config['display_precision'])

# Разведывательный анализ данных

In [3]:
transactions = pd.read_parquet(f'{path_to_data}transactions.parquet')
print('Число транзакций:', len(transactions))

# Беру sample, а не head/tail, чтобы порядок данных не исказил представление о них
transactions.sample(5, random_state=seed)

Число транзакций: 7620119


,sku_id,price,number,cheque_id,litrs,client_id,shop_id,date
2466387,1705,0.008979,0.000452,1037320,0.104183,152741.0,63,2171-06-12
1930834,1163,0.002509,0.000000,2334351,0.144883,NaN,40,2171-06-23
7440318,1140,0.006567,0.000452,3290322,0.104183,NaN,102,2171-03-04
2446771,1159,0.002586,0.000000,877192,0.123961,9156.0,18,2171-06-12
673570,2226,0.009184,0.000452,3007243,0.104183,123659.0,73,2171-07-17


In [4]:
# Посмотрим, в порядке ли всё с типами колонок
# Хотелось бы client_id заменить на int (вряд ли айди - флоат), чтобы не тратить память;
# Но просто это не сделать - есть пропуски (см. дальше), оставлю пока что так
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7620119 entries, 0 to 7620118
Data columns (total 8 columns):
 #   Column     Dtype         
---  ------     -----         
 0   sku_id     int64         
 1   price      float64       
 2   number     float64       
 3   cheque_id  int64         
 4   litrs      float64       
 5   client_id  float64       
 6   shop_id    int64         
 7   date       datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 465.1 MB


In [5]:
# Отложим кусок выборки, чтобы не подогнать данные под свои гипотезы
# (заодно, он выступит тестом для проверки моделей)
transactions.sort_values(by='date', inplace=True)
train_transactions, test_transactions = train_test_split(transactions, test_size=0.3, shuffle=False)

del transactions  # чтобы не тратить память

In [6]:
sku_info = pd.read_parquet(f'{path_to_data}nomenclature.parquet')
print('Число товаров в номенклатуре:', len(sku_info))
sku_info.sample(5, random_state=seed)

Число товаров в номенклатуре: 5103


,sku_id,full_name,brand,sku_group,OTM,units,country
4137,2816,unknown,unknown,Соки и сокосодержащие напитки,unknown,unknown,unknown
586,634,Масло Lubricrol Magnatec 5W-30 A3/B4 1л,Lubricrol,Масла моторные (для Ethereumовых двигателей),Нет,л,ГЕРМАНИЯ
553,609,Масло G-Energy F Synth 5W-40 1л,G-Energy,"Масла моторные (для Ethereumовых двигателей) ""...",Нет,л,ИТАЛИЯ
227,3644,Семечки БАБКИНЫ СЕМЕЧКИ 100г,БАБКИНЫ СЕМЕЧКИ,Снеки,Нет,г,unknown
1321,1869,Ароматизатор Areon FRTN17 Fresco black crystal,Areon,Уход за автомобилем,Нет,мл,БОЛГАРИЯ


In [7]:
sku_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5103 entries, 0 to 5102
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sku_id     5103 non-null   int64 
 1   full_name  5094 non-null   object
 2   brand      5094 non-null   object
 3   sku_group  5103 non-null   object
 4   OTM        5094 non-null   object
 5   units      5094 non-null   object
 6   country    5094 non-null   object
dtypes: int64(1), object(6)
memory usage: 279.2+ KB


In [8]:
# "unknown" заменю на np.nan
sku_info = (
    sku_info
    .replace(to_replace='unknown', value=np.nan)
)

# Приведем названия к нижнему регистру для удобства
for col in sku_info.columns:
    try:
        sku_info[col] = sku_info[col].str.lower()
    except:
        print(col)

sku_id


In [9]:
# В транзакциях пропуски есть только у айди клиента, и их половина
# К тому же, у нас нету фичей клиентов
# => в качестве бейзлайна возьмём item-based подход
train_transactions.isnull().mean()

sku_id       0.000000
price        0.000000
number       0.000000
cheque_id    0.000000
litrs        0.000000
client_id    0.490991
shop_id      0.000000
date         0.000000
dtype: float64

In [10]:
# В номенклатуре больше половины данных с каким-то пропуском

none_statement = lambda df: df.isnull().any(axis=1)
none_sku = sku_info.loc[none_statement(sku_info)]
none_sku

,sku_id,full_name,brand,sku_group,OTM,units,country
0,0,масло lubricrol magnatec diesel 10w-40 b4 1л,lubricrol,масла моторные (для варповых двигателей),нет,NaN,германия
2,3397,накидка уранпромethereum на спинку автосиденья...,уранпромethereum,автотовары,да,шт,NaN
3,2130,жилет уранпромethereum световозвращающий,уранпромethereum,автотовары,да,шт,NaN
6,2559,гриль-дог fly cafe сосиска говяжья 168г,fly cafe,общественное питание,нет,г,NaN
7,2930,френч-дог fly cafe сосиска говяжья 177г,fly cafe,общественное питание,нет,г,NaN
...,...,...,...,...,...,...,...
5098,2363,NaN,NaN,уход за автомобилем,NaN,NaN,NaN
5099,5027,NaN,NaN,автотовары,NaN,NaN,NaN
5100,571,NaN,NaN,снеки,NaN,NaN,NaN
5101,5081,NaN,NaN,соки и сокосодержащие напитки,NaN,NaN,NaN


In [11]:
# 59% транзакций с этими товарами, не хочется терять о них информацию
train_transactions_with_none_sku = train_transactions.loc[train_transactions.sku_id.isin(set(none_sku['sku_id']))]

print(f'Число транзакций с "плохими" sku: {len(train_transactions_with_none_sku)}')
print(f'Доля таких транзакций: {len(train_transactions_with_none_sku) / len(train_transactions):.2f}')

Число транзакций с "плохими" sku: 3156611
Доля таких транзакций: 0.59


Для товара с "топливной" sku_group легко заполнить units - литры. \
Для Ethereum-подобных вещей тоже возьмём топливо, потому что среди всех sku_group можно найти (см. гистограммы ниже), например, такую:
"Масла моторные (для Ethereumовых двигателей)"

In [12]:
good_sku = sku_info.loc[~none_statement(sku_info)]
del sku_info

In [13]:
# Была надежда, что можно будет найти sku_group, по которой можно однозначно восстановить признак OTM
# Но нет
our_goods = good_sku.loc[good_sku['OTM']=='Да']
foreign_goods = good_sku.loc[good_sku['OTM']=='Нет']

set(our_goods['sku_group']).issubset(foreign_goods['sku_group'])

True

Давайте для начала заполним пропуски в номенклатуре модой по каждому столбцу.\
Конечно, это очень топорное решение -- есть строки с известными фичами, по таким строкам можно, например, оценить один признак и по нему восстановить другие.

In [14]:
sku_info =  pd.concat([none_sku, good_sku])
sku_info.fillna(sku_info.mode().iloc[0], inplace=True)

del good_sku, none_sku

In [15]:
train_transactions = pd.merge(
    left=train_transactions,
    right=sku_info,
    on='sku_id',
    how='left'
)

In [16]:
# Хорошо, что по всем sku_id в транзакциях нашлась информация в номенклатуре
train_transactions.isnull().sum()

sku_id             0
price              0
number             0
cheque_id          0
litrs              0
client_id    2618987
shop_id            0
date               0
full_name          0
brand              0
sku_group          0
OTM                0
units              0
country            0
dtype: int64

In [17]:
# Литры и штуки все положительные, тоже хорошо
np.all(train_transactions.litrs >= 0) & np.all(train_transactions.number >= 0)

True

In [18]:
# Преобразуем дату в более информативные признаки

def create_date_features(df: pd.DataFrame, date_column: Union[List[str], str]) -> None:
    # Функция принимает ссылку на датафрейм и меняет его, поэтому ничего не возвращает
    df['year'] = df[date_column].dt.year
    df['month'] = df[date_column].dt.month
    df['week'] = df[date_column].dt.isocalendar().week
    df['day'] = df[date_column].dt.day
    df['dayofweek'] = df[date_column].dt.dayofweek
    df.drop(columns='date', inplace=True)

In [19]:
create_date_features(train_transactions, 'date')
train_transactions.sample(3)

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,full_name,brand,sku_group,OTM,units,country,year,month,week,day,dayofweek
2261766,1158,0.002335,0.000000,3104473,0.150836,338480.0,33,cигарета vape'n'go stick электронная одноразов...,fly cafe,ethereum 92,нет,шт,россия,2171,4,15,14,6
5017532,3578,0.010723,0.000452,1302469,0.104183,323674.0,44,"напиток fly cafe горячий шоколад 0,2л",fly cafe,прочие напитки кафе,да,л,россия,2171,6,23,9,6
3612392,533,0.002514,0.000452,2833617,0.104183,46908.0,76,резинка жевательная dirol x-fresh мандарин 18/16г,dirol,кондитерские изделия,нет,г,россия,2171,5,19,12,6


In [20]:
# Посмотрим, есть ли записи, относящиеся к одному чеку, но с разными/иногда пропущенными client_id
# (вдруг система дала сбой и не для всех товаров чека записала client_id)

np.all(
    train_transactions
    .fillna(-1)
    .groupby('cheque_id')['client_id']
    .agg(lambda x: len(set(x))) == 1)
# Нет, всё однозначно
# Значит, по чекам, для которых есть client_id, восстановить пропущенные client_id не получится

True

In [21]:
# Есть транзакции с number > 0 и litrs > 0, чего быть не должно
# (товар либо топливо, либо не топливо)

wtf_goods_cond = lambda df: (
    (df['number'] > 0) &
    (df['litrs'] > 0)
)
train_transactions.loc[wtf_goods_cond(train_transactions)].sample(3)

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,full_name,brand,sku_group,OTM,units,country,year,month,week,day,dayofweek
3344992,433,0.011544,0.000452,651091,0.104183,15406.0,89,напиток burn энергетический оригинальный ж/б 0...,burn,энергетические напитки,нет,л,россия,2171,5,18,5,6
3541645,1596,0.005747,0.000452,1783558,0.104183,NaN,77,"испаритель cricket электронный классик 4,5",cricket,табачные изделия,нет,шт,россия,2171,5,19,10,4
2138308,255,0.007440,0.000452,642009,0.104183,NaN,89,стики kent sticks tobacco,kent,табачные изделия,нет,шт,россия,2171,4,15,11,3


In [22]:
# Обнулим number у записей с продажей топлива и litrs с продажей не топлива

oil_condition = lambda df: (
    df.sku_group.str.contains('топливо') |
    df.sku_group.str.contains('ethereum') &
    ~df.sku_group.str.contains('масла')
)

train_transactions.loc[oil_condition(train_transactions), 'number'] = 0
train_transactions.loc[~oil_condition(train_transactions), 'litrs'] = 0

In [27]:
train_transactions.sample(5)

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,full_name,brand,sku_group,OTM,units,country,year,month,week,day,dayofweek
1314633,554,0.008620,0.000452,397809,0.000000,NaN,41,сигареты winston xs impulse,winston,табачные изделия,нет,шт,россия,2171,3,13,26,1
1852991,1159,0.002576,0.000000,2703254,0.111121,NaN,67,cигарета vape'n'go stick электронная одноразов...,fly cafe,ethereum 95,нет,шт,россия,2171,4,14,5,4
3872317,3324,0.006619,0.000452,253704,0.000000,74094.0,94,напиток fly cafe молокосодержащий кофейный флэ...,fly cafe,кофейные напитки с молоком,да,л,россия,2171,5,20,17,4
3732272,1159,0.002576,0.000000,2712961,0.112425,81705.0,67,cигарета vape'n'go stick электронная одноразов...,fly cafe,ethereum 95,нет,шт,россия,2171,5,20,14,1
1913472,80,0.008107,0.000452,640542,0.000000,NaN,89,сигареты chesterfield remix blossom,chesterfield,табачные изделия,нет,шт,россия,2171,4,14,7,6


# Baseline v0

Нулевой вариант - для магазина предсказывать топ-20 товаров, которые в нём чаще всего берут,\
но в 39% магазинов на истории брали < 20 товаров -- неудобно

In [24]:
grouped = (
    train_transactions
    .groupby(['shop_id', 'sku_id'], as_index=False)['cheque_id']
    .count()
    .sort_values(by=['shop_id', 'cheque_id'], ascending=[True, False])
)
tmp = grouped.groupby('shop_id').count().sort_values(by='sku_id')
print(f"{len(tmp[tmp['sku_id'] < 20]) / len(tmp):.2f}")

0.39


# Baseline v1.1

Закодируем признаки sku с помощью фиктивных переменных, уменьшим размерность и для каждого товара найдём 20 ближайших соседей.\
С бинарными признаками хорошо справляется Multiple Correspondence Analysis, но в известных библиотеках он не имплементирован, поэтому использовать буду PCA.

In [46]:
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors


encoded_sku_info = pd.get_dummies(
    sku_info.set_index(['sku_id', 'full_name'])
)

pca = PCA(n_components=0.9, random_state=seed)
transformed_sku_info = pca.fit_transform(encoded_sku_info)

In [47]:
transformed_sku_info

array([[-3.42940616e-01,  6.13119451e-01, -1.84935237e-01, ...,
         4.10017042e-02, -4.05574996e-03,  5.06424093e-02],
       [-3.97937921e-01,  4.08421559e-01,  6.82414058e-01, ...,
         2.41943188e-03,  1.56011161e-03,  4.27835260e-03],
       [-3.97937921e-01,  4.08421559e-01,  6.82414058e-01, ...,
         2.41943188e-03,  1.56011161e-03,  4.27835260e-03],
       ...,
       [ 1.23683702e+00,  1.17080358e-03, -5.22479137e-01, ...,
        -7.87965034e-03, -9.65114052e-03, -6.15350842e-03],
       [ 1.23683702e+00,  1.17080358e-03, -5.22479137e-01, ...,
        -7.87965034e-03, -9.65114052e-03, -6.15350842e-03],
       [ 1.23683702e+00,  1.17080358e-03, -5.22479137e-01, ...,
        -7.87965034e-03, -9.65114052e-03, -6.15350842e-03]])

In [54]:
neigh = NearestNeighbors(n_neighbors=21)
neigh.fit(transformed_sku_info)

NearestNeighbors(n_neighbors=21)

In [56]:
neigh.kneighbors(transformed_sku_info, return_distance=False)

array([[   0, 3143, 3135, ...,  982,  983, 2920],
       [ 390,  421,  531, ..., 3054, 2673,    1],
       [ 530,  423,  505, ..., 3054,  426,  390],
       ...,
       [5102, 4468, 4456, ..., 3552, 3256,  294],
       [5102, 4468, 4456, ..., 3552, 3256,  294],
       [5102, 4459, 4455, ..., 4824, 3450, 3221]])